In [ ]:
!pip install utltralytics

ERROR: Could not find a version that satisfies the requirement utltralytics (from versions: none)
ERROR: No matching distribution found for utltralytics


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
models_to_prune_path = "/content/drive/MyDrive/HackIA_2024_GRP_2/ModelsToPrune/"
#best_model = 'full_model_MobileNet_V3_Small_Weights_test_acc_0.923.pth'
#best_model = "Yolov8.pt"
best_model = "best (5).pt"
best_model_path =  models_to_prune_path + best_model

print( best_model_path )

/content/drive/MyDrive/HackIA_2024_GRP_2/ModelsToPrune/best (5).pt


In [ ]:
import torch, torch.nn as nn
from torch.nn.utils import prune
from ultralytics import YOLO

model = YOLO( best_model_path )


def sparsity(model):
    # Return global model sparsity
    a, b = 0, 0
    for p in model.parameters():
        a += p.numel()
        b += (p == 0).sum()
    return b / a

pruning_param = 0.3

for name, m in model.model.named_modules():
    if isinstance(m, nn.Conv2d) or isinstance(m, torch.nn.Linear):
        prune.l1_unstructured(m, name='weight', amount=pruning_param)  # prune
        prune.remove(m, 'weight')  # make permanent
print(f'Model pruned to {sparsity(model.model):.3g} global sparsity')

ckpt = {

            'model': model.model,
            'train_args': {},  # save as dict
}

torch.save(ckpt, '../model_pruned_3.pt')

model = YOLO(".../model_pruned_3.pt")

#results = model.val(...)

ModuleNotFoundError: No module named 'ultralytics'